## Imports

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

import math
import numpy as np
import matplotlib.pyplot as plt

## Download dataset with tfds

In [2]:
(training_set, validation_set), dataset_info = tfds.load('tf_flowers',
                                                          download=True,
                                                          as_supervised=True,
                                                          with_info=True,
                                                          split=['train[:80%]', 'train[80%:]'])

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.0. Subsequent calls will reuse this data.


In [3]:
print(f"Number of training examples : {dataset_info.splits['train'].num_examples}")
print(f"Number of classes : {dataset_info.features['label'].num_classes}")

Number of training examples : 3670
Number of classes : 5


In [4]:
num_examples = dataset_info.splits['train'].num_examples
num_classes = dataset_info.features['label'].num_classes

## Reformat Images and create batches

In [5]:
IMG_SIZE = 224
BATCH_SIZE = 32

In [6]:
def format_images(image, label):
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE)) / 255.0
  return image, label

In [7]:
train_batch = training_set.cache().map(format_images).shuffle(num_examples//4).batch(BATCH_SIZE).prefetch(1)

In [8]:
validation_batch = validation_set.cache().map(format_images).batch(BATCH_SIZE).prefetch(1)

## Download classifier from tensorflow hub

In [11]:
URL = 'https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4'

classifier = hub.KerasLayer(URL, input_shape=(IMG_SIZE, IMG_SIZE, 3))
classifier.trainable = False

## Define a model

In [13]:
model = tf.keras.Sequential([
                             classifier,
                             tf.keras.layers.Dense(num_classes)
])

In [14]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [15]:
EPOCHS = 10

In [16]:
history = model.fit(
    train_batch,
    validation_data=validation_batch,
    epochs=EPOCHS
)

Epoch 1/10
92/92 [==============================] - 8s 85ms/step - loss: 0.7101 - accuracy: 0.7452 - val_loss: 0.4075 - val_accuracy: 0.8678
Epoch 2/10
92/92 [==============================] - 4s 48ms/step - loss: 0.3617 - accuracy: 0.8804 - val_loss: 0.3555 - val_accuracy: 0.8760
Epoch 3/10
92/92 [==============================] - 4s 48ms/step - loss: 0.2875 - accuracy: 0.9026 - val_loss: 0.2994 - val_accuracy: 0.8992
Epoch 4/10
92/92 [==============================] - 4s 48ms/step - loss: 0.2403 - accuracy: 0.9281 - val_loss: 0.2847 - val_accuracy: 0.9046
Epoch 5/10
92/92 [==============================] - 4s 47ms/step - loss: 0.2062 - accuracy: 0.9404 - val_loss: 0.2729 - val_accuracy: 0.9087
Epoch 6/10
92/92 [==============================] - 4s 48ms/step - loss: 0.1840 - accuracy: 0.9489 - val_loss: 0.2665 - val_accuracy: 0.9142
Epoch 7/10
92/92 [==============================] - 4s 48ms/step - loss: 0.1595 - accuracy: 0.9605 - val_loss: 0.2612 - val_accuracy: 0.9128
Epoch 8/10
92